In [ ]:
# from Pianki.Analiza_pianek import saldo, naliczone, wstrzymane, pda
# from Pianki.Analiza_pianek.przygotowanie_danych import aktualizuj_saldo, aktualizuj_zamowienia
# aktualizuj_saldo()
# aktualizuj_zamowienia(pda, "2024-08-12")


In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
from Modele_db.modele_db import ZAM_PIANKI, OWATY, session
import pandas as pd

owaty_w_toku = pd.DataFrame(session.query(ZAM_PIANKI.nr_partii, ZAM_PIANKI.opis, ZAM_PIANKI.ile_zam).filter(
                    ZAM_PIANKI.owaty_kompletacja == None, ZAM_PIANKI.data_zakonczenia == None).all(),
            columns=["NR_PARTII", "OPIS", "ILE_ZAM"]).merge(
    pd.pivot_table(pd.DataFrame(session.query(OWATY.OPIS, OWATY.TYP_OWATY, OWATY.ZUZYCIE).all()), values="ZUZYCIE", columns="TYP_OWATY", index="OPIS", fill_value=0),
    how="left",
    on="OPIS"
).dropna(axis=0)

owaty_w_toku["L1"] = owaty_w_toku.L1 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["O1"] = owaty_w_toku.O1 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["O2"] = owaty_w_toku.O2 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["O3"] = owaty_w_toku.O3 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["W3"] = owaty_w_toku.W3 * owaty_w_toku.ILE_ZAM  
           

ozn_owat = {                  #g/m2, szer, mb
    "O1": ["B/16/150 (1.6x50)", 150, 1.6, 50, "zielona", "73.115.00001"],
    "O2": ["B/16/200 (1.2x40)", 200, 1.2, 40, "niebieska", "73.120.00001"],
    "O3": ["B/16/200 (1.6x40)", 200, 1.6, 40, "czerwona", "73.120.00002"],
}

owaty_podsumowanie = owaty_w_toku.iloc[:,3:].sum()
owaty_Rolki = pd.Series({"L1": 60, 
                         "O1": 50, 
                         "O2": 40, 
                         "O3": 40, 
                         "W3":1000})

owaty = pd.DataFrame([owaty_podsumowanie, owaty_Rolki]).T
owaty.columns = ["ZAPOTRZ", "ROLKI"]
owaty["ILE ROLEK ZPOTRZ"] = owaty.ZAPOTRZ / owaty.ROLKI
owaty


,ZAPOTRZ,ROLKI,ILE ROLEK ZPOTRZ
L1,134.10,60.0,2.23500
O1,1274.15,50.0,25.48300
O2,558.50,40.0,13.96250
O3,2421.55,40.0,60.53875
W3,737.50,1000.0,0.73750


In [7]:
from Modele_db.modele_db import session, ZAM_PIANKI, KOMPLETY_PIANEK
from sqlalchemy import func
from datetime import datetime as dt


session.query(func.sum(ZAM_PIANKI.ile_zam * KOMPLETY_PIANEK.obj)).join(KOMPLETY_PIANEK, KOMPLETY_PIANEK.kod == ZAM_PIANKI.kod).filter(
# session.query(ZAM_PIANKI.opis, ZAM_PIANKI.ile_zam, KOMPLETY_PIANEK.obj).join(KOMPLETY_PIANEK, KOMPLETY_PIANEK.kod == ZAM_PIANKI.kod).filter(
# session.query(ZAM_PIANKI.opis, ZAM_PIANKI.ile_zam).filter(
    ZAM_PIANKI.data_zakonczenia.between(dt(2024,11,3), dt(2024,11,10))).all()


[(86.78499999999998,)]

In [8]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MIN_obj:                     1767
MAX_obj:                     3525
ZAMOWIONE_obj:               836
CZEKA_NA_SPAKOWANIE_obj:     408
CZESCIOWO_DOSTARCZONE_obj:   0
SALDO_obj:                   2302
WOLNE_obj:                   2003
WOLNE_NIE_SPAK_obj:          2411
DO_ZAM_obj:                  514
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 65.3%
WOLNE / MAX: 56.8%
WOLNE_NIE_SPAK / MAX: 68.4%
ZAPEŁNIENIE MAG PO ZDJECIU 2 PACZEK RAZEM Z ZAMOWIENIAMI: 92.1%


In [ ]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"]).drop_duplicates("OPIS")
# br.to_excel("BRAKI.xlsx")

In [4]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 84 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 22 POZYCJE
SALDO PONIZEJ MIN: 70 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 13 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
7,AVANT 2,0.0,0.0,3.0,5,1.0
23,AVANT P64x64,0.0,0.0,7.0,10,7.0
94,"EXTREME 1,5s",0.0,0.0,0.0,8,0.0
97,"EXTREME [1,5s",0.0,0.0,0.0,8,0.0
107,EXTREME P80x105,0.0,0.0,0.0,8,0.0
132,HUDSON [LH,0.0,0.0,6.0,8,6.0
145,MAXWELL P103x92,0.0,0.0,1.0,3,1.0
165,ONYX [LA,0.0,0.0,6.0,8,3.0
197,RITZ 4,0.0,0.0,0.0,4,0.0
219,SPECTRA ][,0.0,0.0,34.0,50,32.0


In [ ]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)

In [ ]:
arp

In [ ]:
arp._Tabela_podsumowania_analizy

In [ ]:
arp["ELI_MIS"].ar

In [ ]:
analiza[analiza.OPIS.str.contains("WIL")][['KOD', 'OPIS', 'MAX','SALDO',
       'ZLECENIA', 'WST', 'CZEKA_NA_SPAKOWANIE', 'ZAMOWIONE',
       'CZESIOWO_DOSTARCZONE', 'MIN', 'SUMA_ZLEC',
       'SALDO_Z_NIE_SPAK', 'WOLNE_SALDO', 'WOLNE_NIE_SPAK', 'DO_ZAM_SZT',
       ]]

In [ ]:
(arp > 1).query("GRUPA < 4")

In [ ]:
arp.Tabela_podsumowania_analizy#.query("GRUPA == 4")

In [ ]:
arp["STONE"].ar

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"

